In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Market parameters
S0 = 100      # Initial stock price
K = 100       # Strike price
T = 1.0       # Time to expiration (1 year)
r = 0.05      # Risk-free rate

# Heston Model parameters
v0 = 0.04     # Initial variance
theta = 0.04  # Long-run variance
kappa = 2.0   # Mean-reversion speed
sigma = 0.2   # Volatility of volatility
rho = -0.7    # Correlation between stock and volatility

# Simulation parameters
N = 252       # Number of time steps (daily)
M = 10000     # Number of simulation paths
dt = T / N    # Time step size

# Set random seed for reproducibility
np.random.seed(42)

# Generate correlated Wiener processes.. got this from online, idk what it does tbh
dW_S = np.random.normal(size=(M, N)) * np.sqrt(dt)
dW_V = rho * dW_S + np.sqrt(1 - rho**2) * np.random.normal(size=(M, N)) * np.sqrt(dt)

# Initialize arrays for stock price and variance
S = np.full((M, N + 1), S0)
V = np.full((M, N + 1), v0)

# Monte Carlo simulation
for t in range(1, N + 1):
    V[:, t] = np.maximum(V[:, t-1] + kappa * (theta - V[:, t-1]) * dt + sigma * np.sqrt(V[:, t-1]) * dW_V[:, t-1], 0)
    S[:, t] = S[:, t-1] * np.exp((r - 0.5 * V[:, t-1]) * dt + np.sqrt(V[:, t-1]) * dW_S[:, t-1])

# Compute  Call Option Payoff
payoff = np.maximum(S[:, -1] - K, 0)

# Discount to present value
option_price = np.exp(-r * T) * np.mean(payoff)

print(f"Estimated Call Option Price: {option_price:.2f}")

plt.figure(figsize=(10, 5))
plt.plot(S[:50, :].T, alpha=0.5)  # Plot first 50 simulations
plt.xlabel("Days")
plt.ylabel("Stock Price")
plt.title("Monte Carlo Simulated Stock Price Paths")
plt.show()